In [19]:
import time
import numpy as np
import math as math
from itertools import compress
from functools import reduce
from random import randint
from math import pi,sqrt,cos,sin

In [30]:
N = 10 #Tamanho da vizinhanca
NumeroItens = 250
MaxWeight = 10.0
MinWeight = 1.0

#Gerar array de itens possiveis
#Pesos reais
#itens = np.random.uniform(low=MinWeight,high = MaxWeight, size=(NumeroItens,))

#Pesos inteiros
itens = np.random.randint(low=MinWeight,high = MaxWeight, size=(NumeroItens,))
print(itens)

#Capacidade maxima da mochila
C = reduce(lambda x,y : x+y, itens)/2
#Vetor de ganho de cada item. O indice do item corresponde ao respectivo ganho
profits = np.vectorize(lambda x: x + 5)(itens)

#print("Espaço de itens: ",itens)
#print("Ganho dos itens", profits)
#print("Capacidade da mochila: ", C)

[7 3 8 8 1 6 5 1 1 4 4 1 2 9 8 7 7 4 3 9 8 4 8 5 9 7 4 6 6 1 1 7 1 9 9 5 1
 1 4 3 5 4 1 3 1 8 4 7 1 9 4 7 3 9 9 8 2 1 9 5 3 5 8 6 8 7 5 8 1 6 6 3 9 4
 3 1 3 9 7 6 4 1 5 8 4 8 8 3 8 6 9 3 2 7 6 6 9 5 9 2 7 2 9 3 8 9 9 7 9 3 3
 3 8 8 7 2 4 9 9 2 9 4 3 3 7 5 5 2 6 6 4 9 6 7 2 4 8 1 6 3 2 1 2 5 7 2 1 4
 6 8 6 9 5 6 3 5 5 4 3 6 5 3 4 9 6 3 7 7 6 6 2 1 8 6 8 8 9 7 6 6 7 4 3 5 8
 8 5 2 3 1 4 5 6 7 2 4 6 7 6 4 1 5 7 6 1 8 2 4 3 1 5 4 8 5 2 9 5 4 7 5 4 3
 9 3 4 2 4 6 7 8 9 3 9 7 5 9 1 2 8 2 9 1 9 4 2 3 7 5 5 9]


In [21]:
#As funcoes de calculo do objetivo e do peso sao iguais.
#Foram criadas duas por questao de facilidade de leitura dos passos executados

#Calculo Funcao Objetivo
def ObjFun(profits, solution):
    return reduce(lambda x,y: x+y, compress(profits, solution),0)

#Calculo peso de uma solucao
def CalcularPeso(itens,solucao):
    return reduce(lambda x,y: x+y, compress(itens, solucao),0)

#Realiza medicoes consecutivas nos qbits de forma a gerar uma solucao classica
def Medir(qindividuos):
    return np.vectorize(lambda x,y : 1 if (x > np.power(y,2)) else 0)\
                        (np.random.rand(NumeroItens), qindividuos[:,1])

#Aplica N medicoes nos q-bits para gerar as solucoes classicas
def GerarVizinhos(qindividuos, N):
    vizinhos = [np.array(Medir(qindividuos)) for i in range(N)]
    return vizinhos

#Realiza os ajustes necessários para manter as solucoes geradas dentro das restricoes de carga da mochila
def AjustarVizinhos(vizinhos,C):
    novosVizinhos = [np.array(AjustarSolucao(vizinho,C)) for vizinho in vizinhos]
    return novosVizinhos

#Realiza o reparo de uma solucao de forma a respeitar as restrições do problema
#O método utilizado é de reparo guloso Lamarckiano, i.e. remoçao consecutiva de
#itens da mochila até a satisfação das restrições
def AjustarSolucao(solucao,C):
    itensSelected = solucao.nonzero()[0]
    Peso = CalcularPeso(itens, solucao)
    while (Peso > C):
        r = np.random.randint(0,itensSelected.shape[0]-1)
        j = itensSelected[r]
        solucao[j] = 0
        Peso = Peso - itens[j]
        itensSelected = np.delete(itensSelected, r)
    return solucao

#Verifica se alguma solucao da iteração é melhor que o bestFitness até o momento
def NewBestFit(melhorSol, bestFit):
    if (ObjFun(profits,melhorSol) > ObjFun(profits,bestFit)):
        return melhorSol
    return bestFit

#Acha a melhor e a pior solucão dentro das solucoes geradas pela medição da vizinhança
def FindBestWorst(vizinhos):
    tmp = [np.array(ObjFun(profits,vizinho)) for vizinho in vizinhos]
    return (vizinhos[np.argmax(tmp)],vizinhos[np.argmin(tmp)])

#Atualiza a população de q-bits. A lista tabu é considerada dentro da função, durante o loop de aplicação das rotações.
#A aplicação da porta quânica em um q-bit k qualquer é proibido de ser aplicado(tabu) caso ambos os bit k da melhor e pior
#solução da iteração sejam, concomitantemente, 0 ou 1(Função xnor = 1 então é tabu).
def AtualizarQ(piorSol, melhorSol, qindividuos):
    theta = 0.01*pi
    
    for i in range(NumeroItens):
        if (melhorSol[i] ^ piorSol[i]): #True: Movimento não tabu
            modSinal = melhorSol[i] - piorSol[i]
            #Verifica se qk está no primeiro/terceiro quadrante ou segundo/quarto e modifica o sinal de theta de acordo
            if (qindividuos[i,0]*qindividuos[i,1] < 0) : modSinal *= -1
                
            Ugate = np.array([[cos(modSinal*theta), -sin(modSinal*theta)],
                              [sin(modSinal*theta),  cos(modSinal*theta)]])  #Matriz de rotação                
            #Rotação do individuo i
            qindividuos[i,:] = np.dot(Ugate,qindividuos[i,:])
    return qindividuos

In [22]:
qindividuos = np.zeros((NumeroItens,2))
qindividuos.fill(1/sqrt(2))
solucao = Medir(qindividuos)

bestFit = solucao

i = 0
NumIter = 1000

print("Limite da mochila: ", C)
print("Nro de iteracoes: ", NumIter)
print("Numero de itens: ", NumeroItens)
print("Peso Inicial:(Sem Reparo)", CalcularPeso(itens,bestFit))

bestFit = AjustarSolucao(bestFit,C)

print("Peso Inicial:(Com Reparo)", CalcularPeso(itens,bestFit))
print("Ganho Inicial:(Com Reparo)", CalcularPeso(profits,bestFit))

start_time = time.time()

while (i < NumIter):
    i = i + 1 
    vizinhos = GerarVizinhos(qindividuos, N) 
    vizinhos = AjustarVizinhos(vizinhos, C)
    (melhorSolucao,piorSolucao) = FindBestWorst(vizinhos)
    bestFit = NewBestFit(melhorSolucao,bestFit)
    qindividuos = AtualizarQ(melhorSolucao, piorSolucao, qindividuos)
    
print("Tempo : ", (time.time() - start_time))
print("Ganho Melhor Solucao Encontrada: ", ObjFun(profits,bestFit))
print("Peso Melhor Solucao Encontrada: ", CalcularPeso(itens,bestFit))

Limite da mochila:  683.597999608
Nro de iteracoes:  1000
Numero de itens:  250
Peso Inicial:(Sem Reparo) 701.963618481
Peso Inicial:(Com Reparo) 676.26010394
Ganho Inicial:(Com Reparo) 1311.26010394
Tempo :  7.761589527130127
Ganho Melhor Solucao Encontrada:  1533.56822664
Peso Melhor Solucao Encontrada:  683.56822664


In [28]:
m = np.empty([NumeroItens,int(C+1)])
print(m)
for j in range(int(C)):
    m[0, j] = 0

for i in range(1,NumeroItens):
    for j in range(int(C+1)):
        if (itens[i] > j):
            m[i, j] = m[i-1, j]
        else:
            m[i, j] = max(m[i-1, j], m[i-1, j-itens[i]] + profits[i])
                
print(m[NumeroItens-1,int(C)])

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


C:\Users\Pedro\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1406.82280405
